# Coursera Peer-graded Assignment
##### Part 3: Segmenting and Clustering Neighborhoods in Toronto

Import necessary libraries.

In [1]:
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

Get the wikipedia page containing the neighborhood data.

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(wiki_url)

Get the html table containing the neighborhood data.

In [3]:
wikipg = BeautifulSoup(response.text, 'html.parser')
table = wikipg.body.find_all('table', 'wikitable sortable')[0]

Convert html table to pandas dataframe.

In [4]:
data = None
for tr in table.tbody.find_all('tr'):
    # extract headers
    th = tr.find_all('th')
    if len(th) > 0:
        data = {t.text.strip(): [] for t in th}
    else:
        for hdr,entry in zip(data.keys(), tr.find_all('td')):
            data[hdr].append(entry.text.strip())
df = pd.DataFrame(data=data, columns=data.keys())
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove rows from the dataframe where Borough is unassigned.

In [5]:
def set_unassigned_neigh(row):
    data = dict(zip(row.index.values, row.values))
    if data['Neighbourhood'] == 'Not assigned':
        data['Neighbourhood'] = data['Borough']
    return pd.Series(data)

df_assigned = df.loc[df['Borough'] != 'Not assigned']
print('Not assigned' in df_assigned['Borough'].unique())
df_assigned = df_assigned.apply(set_unassigned_neigh, axis=1)
print('Not assigned' in df_assigned['Neighbourhood'].unique())

False
False


In [6]:
df_assigned.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine neighborhoods in a single postal code area.

In [7]:
def join_neighborhoods(row):
    fields = ['Borough', 'Neighbourhood']
    borough = row[fields[0]].values.tolist()[0]
    neighborhoods = ', '.join(row[fields[1]])
    values = [borough, neighborhoods]
    return pd.Series(dict(zip(fields, values)))

df_joined = df_assigned.groupby('Postcode').apply(join_neighborhoods).reset_index()

In [8]:
df_joined.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Display the shape of the dataframe.

In [9]:
print('Number of rows in the dataframe: %d' %df_joined.shape[0])

Number of rows in the dataframe: 103


Import geocoder library needed for getting coordinates.

In [10]:
import geocoder

Since, geocoder is not working. Load Geospatial data from csv.

In [11]:
geodata = pd.read_csv('./Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Define function to get coordinates for a particular postal code.

In [12]:
def get_coordinates(row):
    rowdata = dict(zip(row.index.values, row.values))
    postcode, borough = rowdata['Postcode'],rowdata['Borough']
    rowdata['Latitude'] = geodata.loc[geodata['Postal Code']==postcode, 'Latitude'].values[0]
    rowdata['Longitude'] = geodata.loc[geodata['Postal Code']==postcode, 'Longitude'].values[0]
    return pd.Series(rowdata)

In [13]:
df_geo = df_joined.apply(get_coordinates, axis=1)
df_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Rename headers properly.

In [22]:
df_geo.rename(columns={
    'Postcode': 'PostalCode',
    'Neighbourhood': 'Neighborhood'
}, inplace=True)
df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get boroughs in Toronto.

In [32]:
df_toronto = pd.DataFrame(columns=df_geo.columns)
j = 0
for i, row in df_geo.iterrows():
    if 'Toronto' in row.loc['Borough']:
        df_toronto.loc[j,:] = row
        j += 1

In [34]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6764,-79.293
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.669,-79.3156
3,M4M,East Toronto,Studio District,43.6595,-79.3409
4,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


In [35]:
df_toronto.shape

(38, 5)

Display the boroughs on the map centered at Toronto.

In [62]:
import folium

In [39]:
latitude, longitude = 43.6532, -79.3832
map_neigh = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lon, neigh in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=folium.Popup(str(neigh)),
        color='blue',
        fill=True).add_to(map_neigh)
map_neigh

Cluster the neighborhoods by boroughs.

In [45]:
borough_onehot = pd.get_dummies(df_toronto[['Borough']], prefix='', prefix_sep='')
borough_onehot['Neighborhood'] = df_toronto['Neighborhood']
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
toronto_onehot = borough_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Central Toronto,Downtown Toronto,East Toronto,West Toronto
0,The Beaches,0,0,1,0
1,"The Danforth West, Riverdale",0,0,1,0
2,"The Beaches West, India Bazaar",0,0,1,0
3,Studio District,0,0,1,0
4,Lawrence Park,1,0,0,0


In [51]:
from sklearn.cluster import KMeans

kclusters = 4 # four boroughs
toronto_neigh_clustering = toronto_onehot.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_neigh_clustering)
df_toronto['Cluster Labels'] = kmeans.labels_
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M4E,East Toronto,The Beaches,43.6764,-79.293,0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522,0
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.669,-79.3156,0
3,M4M,East Toronto,Studio District,43.6595,-79.3409,0
4,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,2


Display the clusters on the map.

In [61]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)
colors = ['blue', 'black', 'red', 'green']
iterdata = zip(df_toronto['Latitude'],
               df_toronto['Longitude'],
               df_toronto['Neighborhood'],
               df_toronto['Cluster Labels'])
for lat, lon, poi, cluster in iterdata:
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)
toronto_map

As we can see from the map the central boroughs have dense neighborhoods compared to others boroughs.